<a href="https://colab.research.google.com/github/Linux-Server/AI_Engineering/blob/main/SyntheticDataKit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Reference link : https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Meta_Synthetic_Data_Llama3_2_(3B).ipynb#scrollTo=VrBwG2KT7dam



In [1]:
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm synthetic-data-kit==0.0.3
else:
    try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
    except: get_numpy = "numpy"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade         unsloth {get_vllm} {get_numpy} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
    !uv pip install synthetic-data-kit==0.0.3
!uv pip install transformers==4.55.4
!uv pip install --no-deps trl==0.22.2

In [4]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
    except: get_numpy = "numpy"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
!uv pip install transformers==4.55.4
!uv pip install --no-deps trl==0.22.2

In [1]:
from unsloth.dataprep import SyntheticDataKit

checkpoint = "unsloth/Llama-3.2-3B-Instruct"

generator = SyntheticDataKit(model_name=checkpoint, max_seq_length=2048)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 10-01 13:21:44 [__init__.py:244] Automatically detected platform cuda.
ERROR 10-01 13:21:45 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 10-01 13:21:52 [vllm_utils.py:689] Unsloth: Patching vLLM v1 graph capture
INFO 10-01 13:21:52 [vllm_utils.py:717] Unsloth: Patching vLLM v0 graph capture
Unsloth: Using dtype = torch.float16 for vLLM.
Unsloth: vLLM loading unsloth/Llama-3.2-3B-Instruct with actual GPU utilization = 97.33%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 224.
Unsloth: vLLM's KV Cache can use up to 8.36 GB. Also swap space = 0 GB.
vLLM STDOUT: INFO 10-01 13:22:01 [__init__.py:244] Automatically detected platform cuda.
vLLM STDOUT: INFO 

In [9]:
generator.prepare_qa_generation(output_folder="data", temperature=0.7, top_p=0.95, overlap=64, max_generation_tokens=512)

In [12]:
!synthetic-data-kit system-check

vLLM STDOUT: INFO:     127.0.0.1:39196 - "GET /v1/models HTTP/1.1" 200 OK
 VLLM server is running at http://localhost:8000/v1
Available models: {'object': 'list', 'data': [{'id': 
'unsloth/Llama-3.2-3B-Instruct', 'object': 'model', 'created': 1759325716, 
'owned_by': 'vllm', 'root': 'unsloth/Llama-3.2-3B-Instruct', 'parent': None, 
'max_model_len': 2048, 'permission': [{'id': 
'modelperm-f45ab093bfb24898b5b764dc46081f8f', 'object': 'model_permission', 
'created': 1759325716, 'allow_create_engine': False, 'allow_sampling': True, 
'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 
'allow_fine_tuning': False, 'organization': '*', 'group': None, 'is_blocking': 
False}]}]}
⠋ Checking VLLM server at http://localhost:8000/v1...


In [18]:
!synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        ingest "https://www.alongdustyroads.com/posts/kindness-of-kerala"

⠦ Processing https://www.alongdustyroads.com/posts/kindness-of-kerala...
 Text successfully extracted to data/output/www_alongdustyroads_com.txt


In [20]:
# Truncate document
filenames = generator.chunk_data("data/output/www_alongdustyroads_com.txt")
print(len(filenames), filenames[:3])

2 ['data/output/www_alongdustyroads_com_0.txt', 'data/output/www_alongdustyroads_com_1.txt']


In [21]:
import time
# Process 3 chunks for now -> can increase but slower!
for filename in filenames[:3]:
    !synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        create {filename} \
        --num-pairs 25 \
        --type "qa"
    time.sleep(2)

vLLM STDOUT: INFO:     127.0.0.1:39260 - "GET /v1/models HTTP/1.1" 200 OK
vLLM STDOUT: INFO:     127.0.0.1:39262 - "GET /v1/models HTTP/1.1" 200 OK
⠇ Generating qa content from data/output/www_alongdustyroads_com_0.txt...vLLM STDOUT: INFO 10-01 13:56:55 [chat_utils.py:444] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.
vLLM STDOUT: INFO 10-01 13:56:55 [logger.py:43] Received request chatcmpl-01ab07d4273047cc8ff2e1fb0e17c5da: prompt: "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 01 Oct 2025\n\nSummarize this document in 3-5 sentences, focusing on the main topic and key concepts.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nThe Kindness of Kerala — ALONG DUSTY ROADS\n0\nSkip to Content\nHome\nAbout\nBlog\nPlaces\nEverywhere\nEurope\nCentral America & Mexico\nSouth America\nNorth America & Caribbean\nAsia\nAfrica\nALONG DUSTY ROADS\nOpen Me

In [23]:
qa_pairs_filenames = [
    f"data/generated/www_alongdustyroads_com_{i}_qa_pairs.json"
    for i in range(len(filenames[:3]))
]
for filename in qa_pairs_filenames:
    !synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        save-as {filename} -f ft

⠋ Converting data/generated/www_alongdustyroads_com_0_qa_pairs.json to ft format
with json storage...
 Converted to ft format and saved to 
data/final/www_alongdustyroads_com_0_qa_pairs_ft.json
⠋ Converting data/generated/www_alongdustyroads_com_1_qa_pairs.json to ft format
with json storage...
 Converted to ft format and saved to 
data/final/www_alongdustyroads_com_1_qa_pairs_ft.json


In [24]:
from datasets import Dataset
import pandas as pd
final_filenames = [
    f"data/final/www_alongdustyroads_com_{i}_qa_pairs_ft.json"
    for i in range(len(filenames[:3]))
]
conversations = pd.concat([
    pd.read_json(name) for name in final_filenames
]).reset_index(drop = True)

dataset = Dataset.from_pandas(conversations)

In [31]:
dataset

Dataset({
    features: ['messages'],
    num_rows: 27
})

In [33]:
dataset[21]

{'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': 'What made the chai stops an important ritual for the author and Andrew?',
   'role': 'user'},
  {'content': 'the refreshment, chat, people-watching opportunities, or photogenic possibilities',
   'role': 'assistant'}]}

In [34]:
generator.cleanup()

Attempting to terminate the VLLM server gracefully...
vLLM STDOUT: INFO 10-01 14:04:21 [launcher.py:80] Shutting down FastAPI HTTP server.
Server terminated gracefully.
